In [ ]:
import os
import gym
import random
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam,SGD
from collections import deque

In [ ]:
env = gym.make("MountainCar-v0")

In [ ]:
class mountain_agent: 
    
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.gamma = 0.95
        self.lr_rate = 0.005
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_model()
        
    def build_model(self):
        model = Sequential()
        
        model.add(Dense(24,activation='relu',input_dim=self.state_size))
        model.add(Dense(24,activation='relu'))
        model.add(Dense(self.action_size,activation='linear'))
        
        adam = Adam(lr=self.lr_rate)
        sgd = SGD(lr=self.lr_rate)
        model.compile(loss='mse',optimizer=adam)
        
        return model
    
    def rem(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
        
    def act(self,state):
        # Exploration vs Exploitation
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state) # predict reward value based upon current state
        return np.argmax(act_values[0])
    
    def train(self,batch_size=32):
        minibatch = random.sample(self.memory,batch_size)
        
        for state,action,reward,next_state,done in minibatch:
            if not done:
                target = reward + self.gamma*np.argmax(self.target_model.predict(next_state)[0])
            else:
                target = reward
                
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            self.model.fit(state,target_f,epochs=1,verbose=0)
            
        if self.epsilon_min < self.epsilon:
            self.epsilon *= self.epsilon_decay
            
    def update_model(self):
        self.target_model.set_weights(self.model.get_weights())

### Train Model

In [ ]:
state_size = 2
action_size = 3
done=False
ag = mountain_agent(state_size=state_size,action_size=action_size)

In [ ]:
for e in range(200):
    state=env.reset()
    state=np.reshape(state,[1,state_size])
    batch_size=1
    
    for time in range(500):
        env.render()
        action=ag.act(state)
        next_state,reward,done,other_info=env.step(action)
        reward=reward+15 if not done else 1000
        next_state=np.reshape(next_state,[1,state_size])
        ag.rem(state,action,reward,next_state,done)
        state=next_state
        
        if done:
            ag.update_model()
            print(f'Game episode: {e+1}/{100}, High Score: {time}, Exploration rate: {ag.epsilon}')
            print(reward,done,state)
            break
        
    if len(ag.memory)>batch_size:
        ag.train(batch_size)

    
print('Deep Q learner Trained')
env.close()      